### 嘴部、眼部、表情检测

keras == 2.3.1
mediapipe == 0.0.10.1
numpy == 1.21.6
opencv-python == 4.2.0.32
python == 3.7.12
scipy == 1.4.1
tenorflow-gpu == 2.2.0

In [1]:
#mediapipe关键点检测所需package
import cv2
import numpy as np
import mediapipe as mp
from tqdm import tqdm
import time
from scipy.spatial import distance as dist
import pandas as pd
import os
#表情识别所需package
#from keras.preprocessing.image import img_to_array
from keras.utils import image_utils
import imutils
from keras.models import load_model
import math


In [2]:
#表情识别依赖模型配置
emotion_model_path = 'D:/Research/Facedetection/face_mouth_emotion/_mini_XCEPTION.102-0.66.hdf5' #需要修改文件路径

emotion_classifier = load_model(emotion_model_path, compile=False)
EMOTIONS = ["angry" ,"disgust","scared", "happy", "sad", "surprised",
 "neutral"]
 
mp_face_detection = mp.solutions.face_detection 
face_detection = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)

#mediapipe依赖模型配置
mp_face_mesh = mp.solutions.face_mesh
model = mp_face_mesh.FaceMesh(   
        static_image_mode=False,      # 是静态图片还是连续视频帧，摄像头画面为连续视频帧，此处选False
        refine_landmarks=True,       # 使用注意力机制Attention Mesh Model，对嘴唇、眼睛、瞳孔周围的关键点精细定位
        max_num_faces=5,              # 最多检测几张脸
        min_detection_confidence=0.5, # 置信度阈值
        min_tracking_confidence=0.5,  # 追踪阈值
)
# 导入可视化绘图函数
mp_drawing = mp.solutions.drawing_utils 
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1, color=[66,77,229])



mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5,
                                  min_tracking_confidence=0.5)

In [3]:
def rotation_matrix_to_angles(rotation_matrix):
    """
    Calculate Euler angles from rotation matrix.
    :param rotation_matrix: A 3*3 matrix with the following structure
    [Cosz*Cosy  Cosz*Siny*Sinx - Sinz*Cosx  Cosz*Siny*Cosx + Sinz*Sinx]
    [Sinz*Cosy  Sinz*Siny*Sinx + Sinz*Cosx  Sinz*Siny*Cosx - Cosz*Sinx]
    [  -Siny             CosySinx                   Cosy*Cosx         ]
    :return: Angles in degrees for each axis
    """
    x = math.atan2(rotation_matrix[2, 1], rotation_matrix[2, 2])
    y = math.atan2(-rotation_matrix[2, 0], math.sqrt(rotation_matrix[0, 0] ** 2 +
                                                     rotation_matrix[1, 0] ** 2))
    z = math.atan2(rotation_matrix[1, 0], rotation_matrix[0, 0])
    return np.array([x, y, z]) * 180. / math.pi

In [6]:

cap = cv2.VideoCapture(0)
frame_size = (cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_width,frame_height = cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)

while cap.isOpened():
    success, image = cap.read()
    # Convert the color space from BGR to RGB and get Mediapipe results
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image)
    # Convert the color space from RGB to BGR to display well with Opencv
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    face_coordination_in_real_world = np.array([
        [285, 528, 200],
        [285, 371, 152],
        [197, 574, 128],
        [173, 425, 108],
        [360, 574, 128],
        [391, 425, 108]
    ], dtype=np.float64)

    h, w, _ = image.shape
    face_coordination_in_image = []

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            for idx, lm in enumerate(face_landmarks.landmark):
                if idx in [1, 9, 57, 130, 287, 359]:
                    x, y = int(lm.x * w), int(lm.y * h)
                    face_coordination_in_image.append([x, y])

            face_coordination_in_image = np.array(face_coordination_in_image,
                                                  dtype=np.float64)

            # The camera matrix
            focal_length = 1 * w
            cam_matrix = np.array([[focal_length, 0, w / 2],
                                   [0, focal_length, h / 2],
                                   [0, 0, 1]])

            # The Distance Matrix
            dist_matrix = np.zeros((4, 1), dtype=np.float64)

            # Use solvePnP function to get rotation vector
            success, rotation_vec, transition_vec = cv2.solvePnP(
                face_coordination_in_real_world, face_coordination_in_image,
                cam_matrix, dist_matrix)

            # Use Rodrigues function to convert rotation vector to matrix
            rotation_matrix, jacobian = cv2.Rodrigues(rotation_vec)

            result = rotation_matrix_to_angles(rotation_matrix)
            pitch = result[0]
            yaw = result[1]
            roll = result[2]
            for i, info in enumerate(zip(('pitch', 'yaw', 'roll'), result)):
                k, v = info
                #print(k,"here",v)
                text = f'{k}: {int(v)}'
                cv2.putText(image, text, (20, i*30 + 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200, 0, 200), 2)
        F1 = results.multi_face_landmarks[0].landmark[33]; FT_X, FT_Y = int(F1.x * w), int(F1.y * h);   F1 = np.array([FT_X, FT_Y])
        F2 = results.multi_face_landmarks[0].landmark[160]; FT_X, FT_Y = int(F2.x * w), int(F2.y * h);  F2 = np.array([FT_X, FT_Y])
        F3 = results.multi_face_landmarks[0].landmark[159]; FT_X, FT_Y = int(F3.x * w), int(F3.y * h);  F3 = np.array([FT_X, FT_Y])
        F4 = results.multi_face_landmarks[0].landmark[158]; FT_X, FT_Y = int(F4.x * w), int(F4.y * h);  F4 = np.array([FT_X, FT_Y])
        F5 = results.multi_face_landmarks[0].landmark[157]; FT_X, FT_Y = int(F5.x * w), int(F5.y * h);  F5 = np.array([FT_X, FT_Y])
        F6 = results.multi_face_landmarks[0].landmark[133]; FT_X, FT_Y = int(F6.x * w), int(F6.y * h);  F6 = np.array([FT_X, FT_Y])
        F7 = results.multi_face_landmarks[0].landmark[154]; FT_X, FT_Y = int(F7.x * w), int(F7.y * h);  F7 = np.array([FT_X, FT_Y])
        F8 = results.multi_face_landmarks[0].landmark[153]; FT_X, FT_Y = int(F8.x * w), int(F8.y * h);  F8 = np.array([FT_X, FT_Y])
        F9 = results.multi_face_landmarks[0].landmark[145]; FT_X, FT_Y = int(F9.x * w), int(F9.y * h);  F9 = np.array([FT_X, FT_Y])
        F10 = results.multi_face_landmarks[0].landmark[144]; FT_X, FT_Y = int(F10.x * w), int(F10.y * h);  F10 = np.array([FT_X, FT_Y])
        F11 = results.multi_face_landmarks[0].landmark[362]; FT_X, FT_Y = int(F11.x * w), int(F11.y * h);  F11 = np.array([FT_X, FT_Y])
        F12 = results.multi_face_landmarks[0].landmark[384]; FT_X, FT_Y = int(F12.x * w), int(F12.y * h);  F12 = np.array([FT_X, FT_Y])
        F13 = results.multi_face_landmarks[0].landmark[385]; FT_X, FT_Y = int(F13.x * w), int(F13.y * h);  F13 = np.array([FT_X, FT_Y])
        F14 = results.multi_face_landmarks[0].landmark[386]; FT_X, FT_Y = int(F14.x * w), int(F14.y * h);  F14 = np.array([FT_X, FT_Y])
        F15 = results.multi_face_landmarks[0].landmark[387]; FT_X, FT_Y = int(F15.x * w), int(F15.y * h);  F15 = np.array([FT_X, FT_Y])
        F16 = results.multi_face_landmarks[0].landmark[263]; FT_X, FT_Y = int(F16.x * w), int(F16.y * h);  F16 = np.array([FT_X, FT_Y])
        F17 = results.multi_face_landmarks[0].landmark[373]; FT_X, FT_Y = int(F17.x * w), int(F17.y * h);  F17 = np.array([FT_X, FT_Y])
        F18 = results.multi_face_landmarks[0].landmark[374]; FT_X, FT_Y = int(F18.x * w), int(F18.y * h);  F18 = np.array([FT_X, FT_Y])
        F19 = results.multi_face_landmarks[0].landmark[380]; FT_X, FT_Y = int(F19.x * w), int(F19.y * h);  F19 = np.array([FT_X, FT_Y])
        F20 = results.multi_face_landmarks[0].landmark[381]; FT_X, FT_Y = int(F20.x * w), int(F20.y * h);  F20 = np.array([FT_X, FT_Y])
    
        A = dist.euclidean(F2, F10)
        B = dist.euclidean(F3, F9)
        C = dist.euclidean(F4, F8)
        D = dist.euclidean(F5, F7)
        E = dist.euclidean(F1, F6)

    
        F = dist.euclidean(F12, F20)
        G = dist.euclidean(F13, F19)
        H = dist.euclidean(F14, F18)
        I = dist.euclidean(F15, F17)
        J = dist.euclidean(F11, F16)
        
        # ear值
        ear_left = (A + B + C + D) / (4.0 * E)
        ear_right = (F + G + H + I) / (4.0 * J)
        ear = 0.5 * (ear_left + ear_right)

        M1 = results.multi_face_landmarks[0].landmark[62]; FT_X, FT_Y = int(M1.x * w), int(M1.y * h); FT_Color = (31,41,81); M1 = np.array([FT_X, FT_Y])
        M2 = results.multi_face_landmarks[0].landmark[72]; FT_X, FT_Y = int(M2.x * w), int(M2.y * h); FT_Color = (31,41,81); M2 = np.array([FT_X, FT_Y])
        M3 = results.multi_face_landmarks[0].landmark[302]; FT_X, FT_Y = int(M3.x * w), int(M3.y * h); FT_Color = (31,41,81); M3 = np.array([FT_X, FT_Y])
        M4 = results.multi_face_landmarks[0].landmark[293]; FT_X, FT_Y = int(M4.x * w), int(M4.y * h); FT_Color = (31,41,81); M4 = np.array([FT_X, FT_Y])
        M5 = results.multi_face_landmarks[0].landmark[315]; FT_X, FT_Y = int(M5.x * w), int(M5.y * h); FT_Color = (31,41,81); M5 = np.array([FT_X, FT_Y])
        M6 = results.multi_face_landmarks[0].landmark[85]; FT_X, FT_Y = int(M6.x * w), int(M6.y * h); FT_Color = (31,41,81); M6 = np.array([FT_X, FT_Y])
        M7 = results.multi_face_landmarks[0].landmark[11]; FT_X, FT_Y = int(M7.x * w), int(M7.y * h);  M7 = np.array([FT_X, FT_Y])
        M8 = results.multi_face_landmarks[0].landmark[16]; FT_X, FT_Y = int(M8.x * w), int(M8.y * h);  M8 = np.array([FT_X, FT_Y])
    
        K = dist.euclidean(M1, M4)
        L = dist.euclidean(M2, M6)
        M = dist.euclidean(M3, M5)
        N = dist.euclidean(M7, M8)

        
        # mear值
        mear = ( L + M + N) / (3.0 * K)

        # 下面是表情识别部分
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        face_detection_results = face_detection.process(image)
        if face_detection_results.detections:
            for face in face_detection_results.detections:
                face_box=face.location_data.relative_bounding_box

            (fX, fY, fW, fH) = int(face_box.xmin*frame_width),int(face_box.ymin*frame_height),int(face_box.width*frame_width),int(face_box.height*frame_height)
            (fX, fY, fW, fH) = fX-fW//4,fY-fH//4,fW*5//4,fH//4*5
            roi = gray[fY:fY + fH, fX:fX + fW]
            roi = cv2.resize(roi, (64, 64))
            roi = roi.astype("float") / 255.0
            roi = image_utils.img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)

            #表情识别是7分类问题，将概率值最大的作为表情输出
            preds = emotion_classifier.predict(roi)[0]
            emotion_probability = np.max(preds)
            label = EMOTIONS[preds.argmax()]

    print(ear,mear,label,yaw,pitch,roll)
    cv2.imshow('Head Pose Angles', image)

    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()



1/1 [==============================] - 5s 5s/step
0.22117841652606057 0.08299096638287644 neutral 35.29173581724823 -9.703032053253244 [2]
1/1 [==============================] - 0s 14ms/step
0.11502795975810995 0.13961324541900183 sad 18.944256540645927 7.672349998550501 [2]
1/1 [==============================] - 0s 14ms/step
0.08296229802137847 0.11330053032699812 neutral 28.669888591816985 9.226874725700252 [2]
1/1 [==============================] - 0s 15ms/step
0.06863527868306474 0.12010441714572119 sad 34.07947060365627 10.910070395877725 [2]
1/1 [==============================] - 0s 14ms/step
0.07300890084689252 0.13931570326068407 scared 40.710425727757865 12.61260412847682 [2]
1/1 [==============================] - 0s 13ms/step
0.06952080857502937 0.15614751451148146 sad 35.76488913447382 12.336912533011725 [2]
1/1 [==============================] - 0s 14ms/step
0.14850100101560648 0.16979540892241224 sad 23.72500301693268 15.463992735944931 [2]
1/1 [===========================

In [ ]:
root_path = 'D:/Research/EEGdata/li'
video_path = root_path + '/video'
excel_path = root_path + '/faceori'
video_file = os.listdir(video_path)
table=0
for video in video_file:
    table+=1
    input_path=video_path+'/'+video
    #print(input_path)
    #查看导入视频的总帧数
    #input_path = 'C://TJ//detection//video//....mp4' #修改为本地的视频文件地址
    print('视频开始处理',input_path)
        
    # 获取视频总帧数
    cap = cv2.VideoCapture(input_path)
    frame_count = 0
    while(cap.isOpened()):
        success, frame = cap.read()
        frame_count += 1
        if not success:
            break
    cap.release()
    print('视频总帧数为',frame_count)

    #导入视频
    cap = cv2.VideoCapture(input_path)
    frame_size = (cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_width,frame_height = cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)

    # 分为两部分，第一部分获取每一帧画面的关键点，第二部分每一帧画面进行剪裁获取预测表情
    earlist = []
    mearlist = []
    emotionlist = []
    try:
        while(cap.isOpened()):
            success, frame = cap.read()
            if not success:
                    break   

            try:
                h,w = frame.shape[0], frame.shape[1]
                img_RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                # 将RGB图像输入模型，获取预测结果
                results = model.process(img_RGB)
                
                if results.multi_face_landmarks: # 如果检测出人脸

                    F1 = results.multi_face_landmarks[0].landmark[33]; FT_X, FT_Y = int(F1.x * w), int(F1.y * h);   F1 = np.array([FT_X, FT_Y])
                    F2 = results.multi_face_landmarks[0].landmark[160]; FT_X, FT_Y = int(F2.x * w), int(F2.y * h);  F2 = np.array([FT_X, FT_Y])
                    F3 = results.multi_face_landmarks[0].landmark[159]; FT_X, FT_Y = int(F3.x * w), int(F3.y * h);  F3 = np.array([FT_X, FT_Y])
                    F4 = results.multi_face_landmarks[0].landmark[158]; FT_X, FT_Y = int(F4.x * w), int(F4.y * h);  F4 = np.array([FT_X, FT_Y])
                    F5 = results.multi_face_landmarks[0].landmark[157]; FT_X, FT_Y = int(F5.x * w), int(F5.y * h);  F5 = np.array([FT_X, FT_Y])
                    F6 = results.multi_face_landmarks[0].landmark[133]; FT_X, FT_Y = int(F6.x * w), int(F6.y * h);  F6 = np.array([FT_X, FT_Y])
                    F7 = results.multi_face_landmarks[0].landmark[154]; FT_X, FT_Y = int(F7.x * w), int(F7.y * h);  F7 = np.array([FT_X, FT_Y])
                    F8 = results.multi_face_landmarks[0].landmark[153]; FT_X, FT_Y = int(F8.x * w), int(F8.y * h);  F8 = np.array([FT_X, FT_Y])
                    F9 = results.multi_face_landmarks[0].landmark[145]; FT_X, FT_Y = int(F9.x * w), int(F9.y * h);  F9 = np.array([FT_X, FT_Y])
                    F10 = results.multi_face_landmarks[0].landmark[144]; FT_X, FT_Y = int(F10.x * w), int(F10.y * h);  F10 = np.array([FT_X, FT_Y])
                    F11 = results.multi_face_landmarks[0].landmark[362]; FT_X, FT_Y = int(F11.x * w), int(F11.y * h);  F11 = np.array([FT_X, FT_Y])
                    F12 = results.multi_face_landmarks[0].landmark[384]; FT_X, FT_Y = int(F12.x * w), int(F12.y * h);  F12 = np.array([FT_X, FT_Y])
                    F13 = results.multi_face_landmarks[0].landmark[385]; FT_X, FT_Y = int(F13.x * w), int(F13.y * h);  F13 = np.array([FT_X, FT_Y])
                    F14 = results.multi_face_landmarks[0].landmark[386]; FT_X, FT_Y = int(F14.x * w), int(F14.y * h);  F14 = np.array([FT_X, FT_Y])
                    F15 = results.multi_face_landmarks[0].landmark[387]; FT_X, FT_Y = int(F15.x * w), int(F15.y * h);  F15 = np.array([FT_X, FT_Y])
                    F16 = results.multi_face_landmarks[0].landmark[263]; FT_X, FT_Y = int(F16.x * w), int(F16.y * h);  F16 = np.array([FT_X, FT_Y])
                    F17 = results.multi_face_landmarks[0].landmark[373]; FT_X, FT_Y = int(F17.x * w), int(F17.y * h);  F17 = np.array([FT_X, FT_Y])
                    F18 = results.multi_face_landmarks[0].landmark[374]; FT_X, FT_Y = int(F18.x * w), int(F18.y * h);  F18 = np.array([FT_X, FT_Y])
                    F19 = results.multi_face_landmarks[0].landmark[380]; FT_X, FT_Y = int(F19.x * w), int(F19.y * h);  F19 = np.array([FT_X, FT_Y])
                    F20 = results.multi_face_landmarks[0].landmark[381]; FT_X, FT_Y = int(F20.x * w), int(F20.y * h);  F20 = np.array([FT_X, FT_Y])
                
                    A = dist.euclidean(F2, F10)
                    B = dist.euclidean(F3, F9)
                    C = dist.euclidean(F4, F8)
                    D = dist.euclidean(F5, F7)
                    E = dist.euclidean(F1, F6)

                
                    F = dist.euclidean(F12, F20)
                    G = dist.euclidean(F13, F19)
                    H = dist.euclidean(F14, F18)
                    I = dist.euclidean(F15, F17)
                    J = dist.euclidean(F11, F16)
                    
                    # ear值
                    ear_left = (A + B + C + D) / (4.0 * E)
                    ear_right = (F + G + H + I) / (4.0 * J)
                    ear = 0.5 * (ear_left + ear_right)
                    earlist.append(ear)

                    M1 = results.multi_face_landmarks[0].landmark[62]; FT_X, FT_Y = int(M1.x * w), int(M1.y * h); FT_Color = (31,41,81); M1 = np.array([FT_X, FT_Y])
                    M2 = results.multi_face_landmarks[0].landmark[72]; FT_X, FT_Y = int(M2.x * w), int(M2.y * h); FT_Color = (31,41,81); M2 = np.array([FT_X, FT_Y])
                    M3 = results.multi_face_landmarks[0].landmark[302]; FT_X, FT_Y = int(M3.x * w), int(M3.y * h); FT_Color = (31,41,81); M3 = np.array([FT_X, FT_Y])
                    M4 = results.multi_face_landmarks[0].landmark[293]; FT_X, FT_Y = int(M4.x * w), int(M4.y * h); FT_Color = (31,41,81); M4 = np.array([FT_X, FT_Y])
                    M5 = results.multi_face_landmarks[0].landmark[315]; FT_X, FT_Y = int(M5.x * w), int(M5.y * h); FT_Color = (31,41,81); M5 = np.array([FT_X, FT_Y])
                    M6 = results.multi_face_landmarks[0].landmark[85]; FT_X, FT_Y = int(M6.x * w), int(M6.y * h); FT_Color = (31,41,81); M6 = np.array([FT_X, FT_Y])
                    M7 = results.multi_face_landmarks[0].landmark[11]; FT_X, FT_Y = int(M7.x * w), int(M7.y * h);  M7 = np.array([FT_X, FT_Y])
                    M8 = results.multi_face_landmarks[0].landmark[16]; FT_X, FT_Y = int(M8.x * w), int(M8.y * h);  M8 = np.array([FT_X, FT_Y])
                
                    K = dist.euclidean(M1, M4)
                    L = dist.euclidean(M2, M6)
                    M = dist.euclidean(M3, M5)
                    N = dist.euclidean(M7, M8)
                    
                    # mear值
                    mear = ( L + M + N) / (3.0 * K)
                    mearlist.append(mear)
                
                else:
                    mearlist.append("no")
                    earlist.append("no")

                # 下面是表情识别部分
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                face_detection_results = face_detection.process(frame)
                if face_detection_results.detections:
                    for face in face_detection_results.detections:
                        face_box=face.location_data.relative_bounding_box

                    (fX, fY, fW, fH) = int(face_box.xmin*frame_width),int(face_box.ymin*frame_height),int(face_box.width*frame_width),int(face_box.height*frame_height)
                    (fX, fY, fW, fH) = fX-fW//4,fY-fH//4,fW*5//4,fH//4*5
                    roi = gray[fY:fY + fH, fX:fX + fW]
                    roi = cv2.resize(roi, (64, 64))
                    roi = roi.astype("float") / 255.0
                    roi = image_utils.img_to_array(roi)
                    roi = np.expand_dims(roi, axis=0)

                    #表情识别是7分类问题，将概率值最大的作为表情输出
                    preds = emotion_classifier.predict(roi)[0]
                    emotion_probability = np.max(preds)
                    label = EMOTIONS[preds.argmax()]
                    emotionlist.append(label) 
                else:
                    emotionlist.append("no") 
                    continue
            except:
                print('error')
                pass
    except:
        print('中途中断')
        pass

        cv2.destroyAllWindows()
        cap.release()

    output_excel = {'ear':[],'mear':[],'emotion':[]}
    output_excel['ear']=earlist
    output_excel['mear'] = mearlist
    output_excel['emotion'] = emotionlist
    output = pd.DataFrame(output_excel)
    fileName = os.path.splitext(video)[0]
    output.to_csv(excel_path+'/'+fileName+'.csv',index=False)

In [ ]:
import os
import pandas as pd
root_path = 'D:/Research/EEGdata/li'
ori_path =  root_path +'/faceori'
output_path =  root_path + '/facedata_excel'
files = os.listdir(ori_path)
for  excel_file in files:
    input_path=ori_path+'/'+excel_file

    raw = pd.read_csv(input_path)
    raw_data=raw.values
    print(raw)

    datalist={"frame":[],"ear":[],"mear":[],"no":[],"angry":[],"happy":[],"neutral":[],"sad":[],"scared":[],"surprised":[],"disgust":[],"engagement":[]}

    for i in range(len(raw_data)):
        datalist["frame"].append(i+1)
        datalist["ear"].append(raw_data[i][0])
        datalist["mear"].append(raw_data[i][1])
        datalist["no"].append(0)
        datalist["angry"].append(0)
        datalist["happy"].append(0)
        datalist["neutral"].append(0)
        datalist["sad"].append(0)
        datalist["scared"].append(0)
        datalist["surprised"].append(0)
        datalist["disgust"].append(0)
        datalist["engagement"].append(0)
        datalist[raw_data[i][2]][-1]=1
        
    output=pd.DataFrame(datalist)
    fileName = os.path.splitext(excel_file)[0]
    output.to_csv(output_path+'/'+fileName+'.csv',index=False)
